# List of My Movies

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine
from fsUtils import moveFile


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-01-05 18:13:01.084119


In [26]:
import re
from time import sleep
from collections import OrderedDict
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from fsUtils import setDir, isDir, mkDir, setFile, isFile, setSubFile, moveFile
from fileUtils import getBaseFilename, getDirname, getFileBasics
from searchUtils import findSubPatternExt, findPatternExt, findExt, findNearest
from strUtils import convertCurrency
from webUtils import getWebData, getHTML
from numpy import repeat
from movieDB import movieDB
from os import getcwd
from os.path import join
from glob import glob
import operator
from movieRenames import manualRenames

##############################################################################################################################
# Combine Movies
##############################################################################################################################
class mymovies(movieDB):
    def __init__(self, basedir=None):
        self.name = "mymovies"
        movieDB.__init__(self, dbdir=self.name)
        self.combine = None
        
    def setCombinedMovies(self, combine):
        self.combine = combine
        
    def getCombinedMovies(self, debug=False):        
        savename = setFile(self.combine.getResultsDir(), "movies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        combinedMovies = getFile(savename)
        if debug:
            print("Found {0} combined movies".format(len(combinedMovies)))
        return combinedMovies
        
    def getMyMovies(self, debug=False): 
        savename = setFile(self.getDataDir(), "mymovies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        mine = getFile(savename)
        if debug:
            print("Found {0} my movies".format(len(mine)))
        return mine
        
    def findMyMovies(self, debug=False):
        movies = glob("/Volumes/*/Movies/*")
        mine   = dict(zip([getBaseFilename(x) for x in movies], movies))
        print("Found {0} movies on my disks".format(len(movies)))
        savename = setFile(self.getDataDir(), "mymovies.json")
        saveFile(idata=mine, ifile=savename, debug=True)
        
        
    def searchMyMovies(self, movie, year, cutoff=0.8):
        moviesMine = self.getMyMovies()
        key = "{0} [{1}]".format(movie, year)
        result     = findNearest(key, moviesMine.keys(), 3, 0.95)
        if len(result) > 0:
            print("{0}  --->  {1}".format(movie, result))
            location = moviesMine[key]
            print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(location, location))
            return
        else:
            key = "{0} [{1}]".format(movie, year)
            result = findNearest(key, moviesMine.keys(), 3, cutoff)
            if len(result) > 0:
                print("{0}  --->  {1}".format(movie, result))
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(location, location))
                return
        print("No match found")
        return
    
    
    def suggestRenames(self):

        movies = glob("/Volumes/Download/MoviesFinished/*.*")
        for movie in movies:
            fileInfo = getFileBasics(movie)
            name = fileInfo[1]
                        
            ps=["1080", "720", "480"]
            for p in ps:
                name = name.replace("{0}p".format(p), "")

            xs=['264']
            for x in xs:
                name = name.replace("x{0}".format(x), "")
            
            ts=["BluRay", "BRrip", "WEBRip", "HDTVRip", "GAZ", "BrRip"]
            for t in ts:
                name = name.replace("{0}".format(t), "")
                
            vs=["-[YTS.AG]", "-[YTS.AM]", "YIFY", "[YTS.AG]"]
            for v in vs:
                name = name.replace("{0}".format(v), "")
                
            name = name.replace(".", " ")
            name=name.strip()

            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            if name != fileInfo[1]:
                moveFile(src=movie, dst=dst, debug=True)


        movies = glob("/Volumes/Download/MoviesFinished/*.*")   
        for movie in movies:
            fileInfo = getFileBasics(movie)            
            name  = fileInfo[1][:-4].strip()
            year  = fileInfo[1][-4:]
            
            try:
                int(year)
                name = "{0} [{1}]".format(name, year)
            except:
                print("Cannot rename {0}".format(movie))
                continue
            
            src = movie
            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            moveFile(src=src, dst=dst, debug=True)
            print("")
        
        
    def mergeMovies(self, toFile=False, source=None):
        moviesCombined = self.getCombinedMovies()
        moviesMine     = self.getMyMovies()
        renames        = {}
        filename = setFile(self.getResultsDir(), "mymovies.dat")

        if toFile:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"), file=open(filename, "w"))
        else:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"))

        pYear = None
        for key,name in moviesCombined.items():
            movie = key[:-7]
            year  = key[-5:-1]
            
            if source is not None:
                if name != source:
                    continue

            location = moviesMine.get(key)
            if location is None:
                for dYear in [-1,1,-2,2]:
                    dKey = "{0} [{1}]".format(movie, int(year)+dYear)
                    if moviesMine.get(dKey) is not None:
                        location = moviesMine[dKey]
                        break
            if location is None:
                result = findNearest(key, moviesMine.keys(), 1, 0.98)
                if len(result) > 0:
                    location = u'\u2248'
                    myMovie  = moviesMine[result[0]]
                    fileInfo = getFileBasics(myMovie)
                    myMovieYear = fileInfo[1][-5:-1]
                    if abs(int(year) - int(myMovieYear)) <= 0:
                        renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                else:
                    result = findNearest(key, moviesMine.keys(), 1, 0.90)
                    if len(result) > 0:
                        location = u'\u2047'
                        myMovie  = moviesMine[result[0]]
                        fileInfo = getFileBasics(myMovie)
                        myMovieYear = fileInfo[1][-5:-1]
                        if abs(int(year) - int(myMovieYear)) <= 0:
                            renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                    else:
                        location = ""
            else:
                location = u'\u2713'
                
            
            #print(pYear,year)
            if pYear is not None and pYear != year:
                print("\n\n")
            pYear = year
                
            if toFile:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name), file=open(filename, "a"))
            else:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name))

                
                
        print("\n\n\n")
        for key,rename in renames.items():
            if key != rename:
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(key, rename))

In [18]:
mine = mymovies()
mine.setCombinedMovies(combine())
_ = clock("Last Run")

Current Time is Sun Jan 06, 2019 00:35:23 for Last Run


In [19]:
mine.suggestRenames()

In [16]:
mine.findMyMovies()
_ = clock("Last Run")

Found 5107 movies on my disks
Saving data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
Saved data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
  --> This file is 419.8kB.
Current Time is Sun Jan 06, 2019 00:14:13 for Last Run


In [216]:
mine.searchMyMovies("Dr. Strangelove", 1921, cutoff=0.7)

Dr. Strangelove  --->  ['Dr. Strangelove [1964]']


In [20]:
mine.mergeMovies(source="Razzies")

Year  ✓  Movie                                                                                               Source              
1980  ✓  The Blue Lagoon                                                                                     Razzies             
1980     Can't Stop The Music                                                                                Razzies             
1980  ✓  Xanadu                                                                                              Razzies             
1980     The Competition                                                                                     Razzies             
1980     Windows                                                                                             Razzies             
1980  ✓  The Shining                                                                                         Razzies             
1980     Cruising                                                                         

1985     The Bride                                                                                           Razzies             
1985     Night Patrol                                                                                        Razzies             
1985  ✓  James Bond A View To A Kill                                                                         Razzies             
1985     Fever Pitch                                                                                         Razzies             
1985     Revolution                                                                                          Razzies             
1985  ✓  Rambo First Blood Part II                                                                           Razzies             
1985     Perfect                                                                                             Razzies             



1986  ✓  Cobra                                                                         

1991     The Butcher's Wife                                                                                  Razzies             



1992     Final Analysis                                                                                      Razzies             
1992     Shining Through                                                                                     Razzies             
1992     Christopher Columbus The Discovery                                                                  Razzies             
1992  ✓  Stop! Or My Mom Will Shoot                                                                          Razzies             
1992  ✓  Basic Instinct                                                                                      Razzies             
1992     Hoffa                                                                                               Razzies             
1992  ✓  The Bodyguard                                                                 




1999     Crazy In Alabama                                                                                    Razzies             
1999  ✓  Entrapment                                                                                          Razzies             
1999  ✓  The Haunting                                                                                        Razzies             
1999  ✓  Big Daddy                                                                                           Razzies             
1999  ✓  Wild Wild West                                                                                      Razzies             
1999  ✓  The Blair Witch Project                                                                             Razzies             
1999  ✓  For Love Of The Game                                                                                Razzies             
1999  ✓  End Of Days                                                                   

2005     Son Of The Mask                                                                                     Razzies             
2005     Dirty Love                                                                                          Razzies             
2005  ✓  War Of The Worlds                                                                                   Razzies             
2005  ✓  Bewitched                                                                                           Razzies             
2005  ✓  Doom                                                                                                Razzies             
2005     Alone In The Dark                                                                                   Razzies             
2005  ✓  Deuce Bigalow European Gigolo                                                                       Razzies             



2006  ✓  Little Man                                                                    

2014  ✓  Teenage Mutant Ninja Turtles                                                                        Razzies             
2014  ✓  A Million Ways To Die In The West                                                                   Razzies             
2014  ✓  The Legend Of Hercules                                                                              Razzies             
2014  ✓  Transformers IV Age Of Extinction                                                                   Razzies             
2014  ✓  Blended                                                                                             Razzies             
2014  ✓  The Other Woman                                                                                     Razzies             
2014  ✓  Saving Christmas                                                                                    Razzies             
2014  ✓  Left Behind                                                                      

In [27]:
mine.searchMyMovies("BASEketball", 2014, cutoff=0.7)

No match found


In [234]:

moveFile(src="/Volumes/Download/Movies/The Four Horsemen of the Apocalypse [1921].avi", dst="/Volumes/Download/Movies/The Four Horsemen Of The Apocalypse [1921].avi", debug=True)

moveFile(src="/Volumes/Download/Movies/Orphans of the Storm [1921].avi", dst="/Volumes/Download/Movies/Orphans Of The Storm [1921].avi", debug=True)


Moving /Volumes/Download/Movies/The Four Horsemen of the Apocalypse [1921].avi to /Volumes/Download/Movies/The Four Horsemen Of The Apocalypse [1921].avi
  Not moving because /Volumes/Download/Movies/The Four Horsemen Of The Apocalypse [1921].avi exists.
Moving /Volumes/Download/Movies/Orphans of the Storm [1921].avi to /Volumes/Download/Movies/Orphans Of The Storm [1921].avi
  Not moving because /Volumes/Download/Movies/Orphans Of The Storm [1921].avi exists.


In [25]:

moveFile(src="/Volumes/Seagate/Movies/Batman And Robin [1997].mp4", dst="/Volumes/Seagate/Movies/Batman & Robin [1997].mp4", debug=True)


Moving /Volumes/Seagate/Movies/Batman And Robin [1997].mp4 to /Volumes/Seagate/Movies/Batman & Robin [1997].mp4
